In [1]:
%matplotlib inline
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Dropout, Flatten, Dense, Activation, Conv2D, MaxPooling2D
from keras.models import Sequential
import numpy as np
from pak.datasets.CUHK03 import cuhk03
import matplotlib.pyplot as plt
from keras.optimizers import SGD
import json
from pprint import pprint
Settings = json.load(open('../prototyping/settings.txt'))
pprint(Settings)
from pak.datasets.CUHK03 import cuhk03
from CUHK03_Sampler import CUHK03_Sampler

from stacknet import get_model

model = get_model()
model.summary()

Using TensorFlow backend.


{'data_root': '/home/data',
 'deepmatch': '/home/deepmatching_1.2.2_c++/deepmatching-static'}
{'data_root': '/home/data',
 'deepmatch': '/home/deepmatching_1.2.2_c++/deepmatching-static'}
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 112, 112, 64)      3520      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 112, 112, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 56, 56, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 56, 56, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 56, 56, 128)       147584    
_________________________________________________________________
block2_pool (MaxPool

In [2]:
# def generate_training_data():
#     sampler = CUHK03_Sampler()
#     while True:
#         X, Y = sampler.get_train_batch(100, 100)
#         yield X, Y

# model.fit_generator(generate_training_data(), samples_per_epoch=10, 
#                    nb_epoch=10)